In [1]:
import pyspark

## Introduction of pyspark
Anay spark program start with SpartContext, initialized with SparkConf object, 
SparkConf contains all parameters.
getOrCreate will create a new session or use existed session.
SparkSession provide uniform enter point for all API. Like Streaming/SQL/hive..now SparkSession used as enter point of DataFrame and DataSet API(combination of SQL+Hive)

In [3]:
from pyspark import SparkContext as sc
from pyspark import SparkConf
conf = SparkConf().setAppName("miniProject").setMaster('local[*]')
sc=SparkContext.getOrCreate(conf)

### Initialize RDD
for a existed list, use sc.parallelize to create RDD. During operating, the elements in list will be automatically partitioned, and sent to differnt machines.

In [6]:
#1.
rdd = sc.parallelize([1,2,3,4,5])
rdd
#2.Initialized setting by me as 2k, because my laptop is 2-core.
rdd.getNumPartitions()
#3.How is partition?
rdd.glom().collect()
'''
Carefully use collect(), it will exceed the memory
'''

[[1, 2], [3, 4, 5]]

In [19]:
#As suggested by pyspark, install psutil
pip install psutil

SyntaxError: invalid syntax (<ipython-input-19-60b01ccb9234>, line 2)

In [9]:
#4.Another way to create RDD
#Read file in rdd. Each line is an item, be careful aout the path
#1.get work directory
import os
cwd = os.getcwd()
cwd
# 'C:\\Users\\Tianyi Fang\\Desktop\\case\\Untitled Folder'

#2.get the file path
rdd1 = sc.textFile(cwd +"\\20170303_1_play.log.fn")
#rdd_whole_files = sc.wholeTextFiles(cwd + filename)
rdd1

#3.first() will return the first item in rdd
rdd1.first()


'154750654 \tar \t10740680 \t0 \t胸是软绵绵的 \tMC马克 \t76 \t0 \t0 \t 20170303_1_play.log'

### RDD Transformation
1.Basic operations
- map(): do the same operation for every item in RDD
- flatmap():do the same operation for every item in RDD, get a list, then put all results in that list into a new list with flat method
- filter(): filter rows based on conditions
- distinct():
- sample():
- sortBy():
- collect(): transform all item into a py.list, be caucious when dealing with big data

In [14]:
numbersRDD = sc.parallelize(range(1,10+1))
print(numbersRDD.collect())
mapRDD = numbersRDD.map(lambda x: x**2)
print(mapRDD.collect())
filterRDD = numbersRDD.filter(lambda x: x%2 ==0)
print(filterRDD.collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
[2, 4, 6, 8, 10]


In [17]:
sentenceRDD = sc.parallelize(['Hello world!', 'I am learning Spark'])
wordRDD = sentenceRDD.flatMap(lambda sentence: sentence.split(" "))
print(wordRDD.collect(), wordRDD.count())
wordRDD2 = sentenceRDD.map(lambda sentence: sentence.split(" "))
print(wordRDD2.collect(), wordRDD2.count())
'''
Here flatMap represents operations in Python as:
l = ['Hello world!', 'I am learning Spark']
ll = []
for sentence in l:
    ll = ll + sentence.split(" ")
ll
'''

['Hello', 'world!', 'I', 'am', 'learning', 'Spark'] 6
[['Hello', 'world!'], ['I', 'am', 'learning', 'Spark']] 2


2.Use in series

In [18]:
def doubleIfOdd(x):
    if x%2 ==1:
        return 2*x
    else:
        return x
resultRDD = (numbersRDD
            .map(doubleIfOdd)
            .filter(lambda x: x >6)
            .distinct())
resultRDD.collect()

[10, 14, 8, 18]